<a href="https://colab.research.google.com/github/marquesgabi/Doutorado/blob/master/04_ANN_fotos_testa_variaveis_duas_classes_TensorFlow_abr_03_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# rede neural com duas classes grao=2 e nao grao=1+3

In [0]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import cv2
import math
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 
from mpl_toolkits.mplot3d import Axes3D, axes3d
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import matplotlib
import re
%matplotlib inline

In [3]:
!git clone https://github.com/ucfilho/marquesgabi_mar_2020 #clonar do Github
%cd marquesgabi_mar_2020

!git clone https://github.com/marquesgabi/Doutorado
%cd Doutorado

fatal: destination path 'marquesgabi_mar_2020' already exists and is not an empty directory.
/content/marquesgabi_mar_2020
fatal: destination path 'Doutorado' already exists and is not an empty directory.
/content/marquesgabi_mar_2020/Doutorado


In [0]:
#!ls

In [0]:
#@title Digite os parametros { run: "auto", vertical-output: true }
#@markdown Enter neurons hidden layer
N_Hidden=22  #@param {type:"number"}
#@markdown Activation function

Ativa='tanh'#@param ["tanh", "identity", "logistic","relu"] {allow-input: true}

In [0]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [8]:
df_contraste= pd.read_csv('Contraste_Completo_mar_2020.csv') 
#df_contraste= np.delete(df_contraste, 0, axis=1) # delete first colunm
del df_contraste['Unnamed: 0']
Nomes=list(df_contraste.columns)
#del df_contraste['Unnamed: 0'] # delete first colunm
print(Nomes)
print(df_contraste.head())
print(df_contraste.shape)

['Type', 'ASM', 'constrast', 'correl', 'variance', 'inv diff mom', 'sum aveg', 'sum var', 'sum entropy', 'entropy', 'dif var', 'dif entropy', 'IMC1', 'IMC2', 'colors']
   Type      ASM  constrast   correl  ...  dif entropy     IMC1     IMC2  colors
0     1  0.05766   93.16340  0.15346  ...      2.72958 -0.64142  0.98102  0.4125
1     1  0.05013  402.45588  0.32031  ...      2.88069 -0.79959  0.99822  0.4100
2     1  0.05287  154.89461  0.28263  ...      2.93133 -0.74008  0.99509  0.4225
3     1  0.05011  397.55801  0.21760  ...      2.99470 -0.78154  0.99769  0.5300
4     1  0.05408  108.01144  0.34926  ...      2.80948 -0.72303  0.99423  0.4900

[5 rows x 15 columns]
(720, 15)


In [9]:
df_fracionado= pd.read_csv('Contraste_Fracionado_mar_2020.csv') 
del df_fracionado['Unnamed: 0']
Nomes_fracionado=list(df_fracionado.columns)
print(df_fracionado)
#df_fracionado= np.delete(df_fracionado, 0, axis=1) # delete first colunm
#del df_fracionado['Unnamed: 0'] # delete first colunm
#del df_fracionado['Type'] # delete first colunm

print(df_fracionado.head())
#print(df_fracionado.shape)

     N_peaks  Media_proem        Area  ...      Mode       Mean          Sd
0          2     13.64931  1407.70370  ...   7.52778   93.16340   101.41799
1          2    268.50231  6954.06944  ...  36.37500  402.45588   374.61667
2          2     16.58218  2372.18981  ...  46.93750  154.89461   109.03101
3          2      6.75694  5663.77546  ...  77.49306  397.55801   451.88121
4          3     25.91667  1681.69907  ...  10.22917  108.01144    94.42290
..       ...          ...         ...  ...       ...        ...         ...
715        2      2.82755  2325.76157  ...  10.03472  185.27614   346.45698
716        2     10.04282  1243.38426  ...   8.18056   88.03431    95.03825
717        3     12.63426   466.21296  ...   6.06944   29.09273    21.04542
718        2     21.59028  4145.96991  ...  24.84028  563.94812  1900.11526
719        0      0.00000  5489.27083  ...  19.20139  405.43096   487.97502

[720 rows x 10 columns]
   N_peaks  Media_proem        Area  ...      Mode       Mean  

In [10]:
#print(Nomes_contraste)
print(Nomes_fracionado)

['N_peaks', 'Media_proem', 'Area', 'Width_peaks', 'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode', 'Mean', 'Sd']


In [11]:
#ANN_Completo é o dataframe que tem todas as variaveis juntas

ANN_Completo = pd.concat([df_contraste, df_fracionado], axis=1)
print(ANN_Completo.head())
print(ANN_Completo.shape)

   Type      ASM  constrast   correl  ...     Median      Mode       Mean         Sd
0     1  0.05766   93.16340  0.15346  ...   53.48611   7.52778   93.16340  101.41799
1     1  0.05013  402.45588  0.32031  ...  256.77083  36.37500  402.45588  374.61667
2     1  0.05287  154.89461  0.28263  ...  120.70833  46.93750  154.89461  109.03101
3     1  0.05011  397.55801  0.21760  ...  149.81944  77.49306  397.55801  451.88121
4     1  0.05408  108.01144  0.34926  ...   66.18750  10.22917  108.01144   94.42290

[5 rows x 25 columns]
(720, 25)


In [12]:
#Aqui iremos escolher quais variáveis iremos utilizar na ANN

Colunms=['N_peaks', 'Media_proem', 'Area', 'Width_peaks', 'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode','Mean', 'Sd']

#Colunms=['N_peaks', 'Media_proem', 'Area', 'Width_peaks', 
         # 'Width_peaks_max', 'Width_peaks_min', 'Median', 
          #'Mode', 'Mean', 'Sd']

#'ASM', 'constrast', 'correl', 'variance', 'inv diff mom', 'sum aveg', 'sum var', 'sum entropy', 'entropy', 'dif var', 'dif entropy', 'IMC1', 'IMC2', 'colors'
#'N_peaks', 'Media_proem', 'Area', 'Width_peaks', 'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode', 'Mean', 'Sd'


del df_contraste['Type'] # delete first colunm

Nomes_contraste=list(df_contraste.columns)
for ii in Nomes_contraste:
  Colunms.append(ii)
#X=df.iloc[:,2:12]
#y=df.iloc[:,1].astype(int)
X=ANN_Completo[Colunms]
print(X)
#y=df['Type'].astype(int)
y=ANN_Completo['Type']
print(y)

     N_peaks  Media_proem        Area  ...     IMC1     IMC2  colors
0          2     13.64931  1407.70370  ... -0.64142  0.98102  0.4125
1          2    268.50231  6954.06944  ... -0.79959  0.99822  0.4100
2          2     16.58218  2372.18981  ... -0.74008  0.99509  0.4225
3          2      6.75694  5663.77546  ... -0.78154  0.99769  0.5300
4          3     25.91667  1681.69907  ... -0.72303  0.99423  0.4900
..       ...          ...         ...  ...      ...      ...     ...
715        2      2.82755  2325.76157  ... -0.72472  0.99488  0.5300
716        2     10.04282  1243.38426  ... -0.76055  0.99696  0.4400
717        3     12.63426   466.21296  ... -0.71517  0.99219  0.5225
718        2     21.59028  4145.96991  ... -0.71082  0.99428  0.5325
719        0      0.00000  5489.27083  ... -0.73096  0.99265  0.5100

[720 rows x 24 columns]
0      1
1      1
2      1
3      1
4      1
      ..
715    3
716    3
717    3
718    3
719    3
Name: Type, Length: 720, dtype: int64


In [13]:
print(Colunms)

['N_peaks', 'Media_proem', 'Area', 'Width_peaks', 'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode', 'Mean', 'Sd', 'ASM', 'constrast', 'correl', 'variance', 'inv diff mom', 'sum aveg', 'sum var', 'sum entropy', 'entropy', 'dif var', 'dif entropy', 'IMC1', 'IMC2', 'colors']


In [14]:

# separa os dados variaveis de entrada em normatizados e originais
X_OLD=[]
X_OLD=X.copy()
X=Normatiza(X)
#print(X)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
print(y.ravel())

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 

In [16]:
# esta parte encontra os indices do indefinido para depois 
# usar os indicies para unir indefinido e buraco

Ind=[i for i, e in enumerate(y) if e == 3]
print(Ind)

[540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719]


In [17]:
# esta parte une indefinido e buraco no numero 1 e passa a ser uma so 
# classe por enquanto os dados nao estao balanceados 
# tem o dobro da classe 1 que a classe 2
for i in Ind:
  y[i]=1
print(y.ravel())

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
#Podemos colocar essa parte que automaticamente escolhe o número de neuronios da camada oculta nao é?????
'''N=10
M=10
test=Original(y_test,Y_OLD)
resist_obs_test=[]
resist_obs_test=test['Resistence'].copy()
Melhor=1e99 #toda vez que fizer o teste e o valor da iteracao der menor que o erro. r^2 nao costuma ser o melhor para calcular o erro. MSE=1/n(yexp-ycal)^2 
ir=0
jr=0
for i in range(1,N): #numero de neuronios da primeira camada oculta
    for j in range(1,M):
        col_names=list(y_train)
        clf=rede(i,j)
        clf.fit(x_train, y_train)
        y_calc_train=[]
        y_calc_test=[]
        y_calc_train=clf.predict(x_train)
        y_calc_test=clf.predict(x_test)
        y_calc_train=pd.DataFrame(y_calc_train)
        y_calc_test=pd.DataFrame(y_calc_test)
        y_calc_train.columns = col_names
        y_calc_test.columns = col_names
        y_calc_train=Original(y_calc_train,Y_OLD)
        y_calc_test=Original(y_calc_test,Y_OLD)
        resist_calc_test=[]
        resist_calc_test=y_calc_test['Resistence'].copy()
        mse=mean_squared_error(resist_obs_test,resist_calc_test)
        R2=r2_score(resist_obs_test,resist_calc_test)
        print("Neuronios=",i,j,"MSE teste=",mse,"R^2 teste=",R2)
        if(mse<Melhor):
            Melhor=mse.copy()
            ix=i
            jx=j
            R2r=R2
print("\n Melhor resposta:")
print("Neuronios=",ix,jx,"MSE teste=",Melhor,"R^2 teste=",R2r)'''

'N=10\nM=10\ntest=Original(y_test,Y_OLD)\nresist_obs_test=[]\nresist_obs_test=test[\'Resistence\'].copy()\nMelhor=1e99 #toda vez que fizer o teste e o valor da iteracao der menor que o erro. r^2 nao costuma ser o melhor para calcular o erro. MSE=1/n(yexp-ycal)^2 \nir=0\njr=0\nfor i in range(1,N): #numero de neuronios da primeira camada oculta\n    for j in range(1,M):\n        col_names=list(y_train)\n        clf=rede(i,j)\n        clf.fit(x_train, y_train)\n        y_calc_train=[]\n        y_calc_test=[]\n        y_calc_train=clf.predict(x_train)\n        y_calc_test=clf.predict(x_test)\n        y_calc_train=pd.DataFrame(y_calc_train)\n        y_calc_test=pd.DataFrame(y_calc_test)\n        y_calc_train.columns = col_names\n        y_calc_test.columns = col_names\n        y_calc_train=Original(y_calc_train,Y_OLD)\n        y_calc_test=Original(y_calc_test,Y_OLD)\n        resist_calc_test=[]\n        resist_calc_test=y_calc_test[\'Resistence\'].copy()\n        mse=mean_squared_error(resi

In [0]:
# Train Test Split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, 
                                                    #random_state=25)

In [0]:
# define a rede neural 
#N_Hidden=(22,22,22)

#clf = MLPClassifier(solver='lbfgs',activation=Ativa,alpha=1e-5,
                    #hidden_layer_sizes=N_Hidden,random_state=0)


In [20]:
# define a rede neural

mnist = tf.keras.datasets.mnist

(X_train, y_train),(X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5)
model.evaluate(X_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2216 - accuracy: 0.9337
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0976 - accuracy: 0.9696
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0694 - accuracy: 0.9778
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0511 - accuracy: 0.9838
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0717 - accuracy: 0.9775


[0.0717049241065979, 0.9775000214576721]

In [0]:
# treina a rede neural
#clf.fit(X_train, y_train)


In [0]:
# usa a rede neural treina em calculos para treino e teste
#y_calc_train=clf.predict(X_train)
#y_pred=clf.predict(X_test)


In [21]:
# Evaluating the Algorithm
print(confusion_matrix(y, y_test))
print(classification_report(y, y_test))

ValueError: ignored

In [0]:
'''
[[31  5 15]
 [ 8 41 11]
 [12 12 27]]
              precision    recall  f1-score   support

           1       0.61      0.61      0.61        51
           2       0.71      0.68      0.69        60
           3       0.51      0.53      0.52        51

    accuracy                           0.61       162
   macro avg       0.61      0.61      0.61       162
weighted avg       0.61      0.61      0.61       162
'''


In [0]:
print(y_pred)